In [1]:
from platform import python_version
print(python_version())

3.6.12


In [2]:
cvid_url='https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2Fnytimes%2Fcovid-19-data%2Fmaster%2Fus-counties.csv&filename=us-counties.csv'
cvid_file='us-counties.csv'

aqi_url='https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2020.zip'
aqi_zip='daily_aqi_by_county_2020.zip'
aqi_file='daily_aqi_by_county_2020.csv'
# thresh_pcntg = 0.6 # drop parameter if fraction of null is less than this fraction

In [3]:
incubationPeriod=5

## Download data

In [4]:
import requests, zipfile

try:
    r=requests.get(aqi_url)
    with open(aqi_zip,'wb+') as f:
        f.write(r.content) 

except:
    print(f"Couldn't get aqi data from {aqi_url}")


    
try:
    r=requests.get(cvid_url)
    with open(cvid_file,'wb+') as f:
        f.write(r.content)
    with zipfile.ZipFile(aqi_zip, 'r') as zip_ref:
        zip_ref.extractall('./')
except:
    print(f"Couldn't get cvid data from {cvid_url}")

## Read Covid-19 data for US

In [5]:
import pandas as pd

cvid=pd.read_csv(cvid_file)
cvid.county=cvid.county.str.lower()
cvidCounties = cvid.county.unique().tolist()

## Read AQI data

In [6]:
aqi=pd.read_csv(aqi_file)
aqi=aqi.rename(columns={'county Name':'City'})
aqi.City=aqi.City.str.lower()
aqiCounties=aqi.City.unique().tolist()
aqi=aqi.drop(columns=['State Code', 'County Code', 'Category', 'Number of Sites Reporting'])

## Consider Counties which have both Covid data and AQI info available

In [7]:
commonCounties=set(aqiCounties).intersection(cvidCounties)
len(commonCounties)

778

## Keep rows with Counties from Common Counties only

In [8]:
cvid=cvid[cvid.county.isin(commonCounties)]
aqi=aqi[aqi.City.isin(commonCounties)]

## Sync Dates (i.e. keep common dates only)

In [9]:
min(cvid.date), max(cvid.date), min(aqi.Date), max(aqi.Date)

('2020-01-21', '2020-12-05', '2020-01-01', '2020-11-04')

In [10]:
startdate=max(min(cvid.date), min(aqi.Date))
enddate=min(max(cvid.date), max(aqi.Date))
startdate, enddate

('2020-01-21', '2020-11-04')

In [11]:
cvid=cvid[cvid.date.between(startdate, enddate, inclusive=True)]
aqi=aqi[aqi.Date.between(startdate, enddate, inclusive=True)]

## Split dataframe by County name
## Sync rows between cvid and aqi for each county

## FillNa Method

In [14]:
cvidByCounty={county:df for county, df in cvid.groupby('county')}
aqiByCounty={county:df for county, df in aqi.groupby('City')}

cvid_aqiByCounty={}

for county, cvidCounty in cvidByCounty.items():
    
    aqiCounty=aqiByCounty[county]
    
    aqiCountyDates=set(aqiCounty.Date)
    cvidCountyDates=set(cvidCounty.date)
    
    # filtering common dates only
    commonCountyDates=aqiCountyDates.intersection(cvidCountyDates)
    cvidByCounty[county]=cvidCounty[cvidCounty.date.isin(commonCountyDates)]
    aqiByCounty[county]=aqiCounty[aqiCounty.Date.isin(commonCountyDates)]  
    
    
    cvidByCounty[county]=cvidByCounty[county].groupby('date').agg({'cases': 'mean', 
                                            'date': lambda x: pd.unique(x)}).reset_index(drop=True)    
    
    
    aqiByCounty[county]=aqiByCounty[county].drop_duplicates(subset = 'Date', keep = 'last')
    aqiByCounty[county]=aqiByCounty[county].pivot(index='Date', columns='Defining Parameter', values='AQI')
    aqiByCounty[county].reset_index(inplace=True)

    if not len(cvidByCounty[county]):
        continue    
    
    aqiByCounty[county]=aqiByCounty[county].sort_values(by=['Date'])
    cvidByCounty[county]=cvidByCounty[county].sort_values(by=['date'])
    
    
    cvid_aqiByCounty[county]=aqiByCounty[county].copy()
    cvid_aqiByCounty[county]['cases']=cvidByCounty[county]['cases'].tolist()
    cvid_aqiByCounty[county]['cases']=cvid_aqiByCounty[county]['cases'].shift(incubationPeriod)
    cvid_aqiByCounty[county]=cvid_aqiByCounty[county].iloc[incubationPeriod:]
    
    
#     cvid_aqiByCounty[county]=cvid_aqiByCounty[county].fillna(cvid_aqiByCounty[county].mean())
    cvid_aqiByCounty[county]=cvid_aqiByCounty[county].dropna(axis=0, how='any')
    cvid_aqiByCounty[county].reset_index(drop=True, inplace=True)
    
    if not len(cvid_aqiByCounty[county]):
        cvid_aqiByCounty.pop(county)
        continue

    cvid_aqiByCounty[county]=cvid_aqiByCounty[county].sort_values(['Date'])
    temp=pd.Series([0] + cvid_aqiByCounty[county].cases.tolist()[:-1]).to_numpy()
    cvid_aqiByCounty[county].cases=cvid_aqiByCounty[county].cases.to_numpy() - temp
    print(county, len(cvidByCounty[county]), len(aqiByCounty[county]), len(cvid_aqiByCounty[county]))
    
    print(cvid_aqiByCounty[county].isna().sum())

aiken 179 179 174
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
alexandria city 55 55 50
Defining Parameter
Date     0
PM10     0
cases    0
dtype: int64
allegany 85 85 80
Defining Parameter
Date     0
SO2      0
cases    0
dtype: int64
amador 134 134 129
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
anderson 235 235 230
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
andrew 112 112 107
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
archuleta 172 172 167
Defining Parameter
Date     0
PM10     0
cases    0
dtype: int64
arkansas 61 61 56
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype: int64
ashley 62 62 57
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype: int64
ashtabula 195 195 190
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
asotin 151 151 146
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype: int64
atascosa 16 16 11
Defining Parameter
Date     0
PM2.5    0
cases    

gunnison 165 165 160
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
hall 228 228 223
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype: int64
hanover 203 203 198
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
harney 6 6 1
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype: int64
henderson 102 102 97
Defining Parameter
Date     0
SO2      0
cases    0
dtype: int64
hendricks 238 238 233
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
highlands 99 99 94
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
hood 123 123 118
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
hopewell city 28 28 23
Defining Parameter
Date     0
PM10     0
cases    0
dtype: int64
horry 151 151 146
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
houston 166 166 161
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype: int64
hughes 99 99 94
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype:

salem city 39 39 34
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype: int64
san miguel 61 61 56
Defining Parameter
Date     0
PM10     0
cases    0
dtype: int64
sandoval 107 107 102
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
saratoga 194 194 189
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
sarpy 163 163 158
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype: int64
scotts bluff 192 192 187
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype: int64
sevier 168 168 163
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
sheboygan 100 100 95
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
sherman 87 87 82
Defining Parameter
Date     0
PM10     0
cases    0
dtype: int64
simpson 101 101 96
Defining Parameter
Date     0
Ozone    0
cases    0
dtype: int64
snohomish 224 224 219
Defining Parameter
Date     0
PM2.5    0
cases    0
dtype: int64
somerset 169 169 164
Defining Parameter
Date     0
Ozone    0


## Concatenate all County data
## FillNa

In [16]:
df=pd.concat(list(cvid_aqiByCounty.values()))
# df=df.sample(frac=0.7)
# df.fillna(df.mean(), inplace=True)

df['AQI']=df[['Ozone', 'PM10', 'SO2', 'PM2.5', 'NO2']].max(axis=1)
df['cases']=df['cases']
df.isnull().sum()
len(df)

32948

## Initialize the model

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler

model=LinearRegression()

## Train and Test

In [18]:
X, y = df[['AQI']], df.cases
y=y.to_numpy().reshape(-1, 1)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

poly=PolynomialFeatures(2)
X_train=poly.fit_transform(X_train)
X_test = poly.transform(X_test)

In [22]:
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print(r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred))

-7.392632338509841e-05 3059.9344969342933


## Save Predictions

In [23]:
import numpy as np
df['risk_predicted']=MinMaxScaler().fit_transform(np.log10(model.predict(poly.transform(X)+1)))
df[['cases', 'AQI', 'risk_predicted']].to_csv('validation.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log10
  


## Save Regressor

In [24]:
import pickle

filename = 'regression_model.sav'
pickle.dump(model, open(filename, 'wb+'))

## Predict and Save

In [25]:
predictor=pickle.load(open(filename, 'rb'))

In [27]:
import curr_aqi, time

loc_list = list(cvid_aqiByCounty.keys())
APIDF = curr_aqi.show_aqi(loc_list)
APIDF

,county,AQI,CO,NO2,Ozone,PM2.5,SO2
0,aiken,NaN,NaN,NaN,NaN,NaN,NaN
1,alexandria city,14.0,0.1,1.0,11.8,14.0,0.6
2,allegany,NaN,NaN,NaN,NaN,NaN,NaN
3,amador,72.0,7.8,23.2,NaN,72.0,NaN
4,anderson,38.0,NaN,NaN,NaN,38.0,NaN
5,andrew,NaN,NaN,NaN,NaN,NaN,NaN
6,archuleta,NaN,NaN,NaN,NaN,NaN,NaN
7,arkansas,NaN,NaN,NaN,NaN,NaN,NaN
8,ashley,NaN,NaN,NaN,NaN,NaN,NaN
9,ashtabula,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
params=list(set(['Ozone', 'PM10', 'SO2', 'PM2.5', 'NO2']).intersection(APIDF.columns))

APIDF['AQI']=APIDF[params].max(axis=1)
APIDF=APIDF[~APIDF['AQI'].isna()]
resultX=APIDF['AQI'].to_numpy().reshape(-1, 1)
APIDF['risk_predicted']=MinMaxScaler().fit_transform(np.log10(predictor.predict(poly.transform(resultX)+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Generate color

In [56]:
# import colorsys

# def get_color(val):
#     # suyash code the logic
#     return "colorhexcode"

In [57]:
# APIDF['color']=APIDF['risk_predicted'].apply(lambda x: get_color(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Save as sqlite3

In [58]:
import sqlite3
cnx = sqlite3.connect('results.db')
APIDF.to_sql(name='main', con=cnx, if_exists='replace')
cnx.close()

In [59]:
APIDF

,county,AQI,CO,NO2,Ozone,PM2.5,SO2,risk_predicted,color
1,alexandria city,14.0,0.1,1.0,11.8,14.0,0.6,0.000000,"[255, 73, 73]"
3,amador,72.0,7.8,23.2,NaN,72.0,NaN,0.960682,"[157, 45, 45]"
4,anderson,38.0,NaN,NaN,NaN,38.0,NaN,1.000000,"[153, 44, 44]"
